# Empirical Asset Pricing - PS5

Maximilian Huber

## Task 1


In [1]:
using CSV, DataFrames, Query, Plots, Optim; gr();

### (a)

In [2]:
data = CSV.read("./Data/PS5.csv", delim='\t')
data[:, 1] = Date.(data[:, 1], "mm/dd/yyyy")
data[:yearmonth] = Dates.yearmonth.((data[:, 1]))
data = by(data, :yearmonth) do df
    return df[indmax(Dates.day.(df[:Date])), 2:end-1] 
end;

### (b)

In [3]:
data = hcat(data, 
    DataFrame(
        hcat([n * data[:, 1+n] - (n - 1) * data[:, n] for n in 2:10]...), 
        Symbol.(["fwd$(y)y" for y in 2:10])));

In [4]:
head(data)

,yearmonth,SVENY01,SVENY02,SVENY03,SVENY04,SVENY05,SVENY06,SVENY07,SVENY08,SVENY09,SVENY10,fwd2y,fwd3y,fwd4y,fwd5y,fwd6y,fwd7y,fwd8y,fwd9y,fwd10y
1,"(2018, 2)",2.0493,2.2631,2.4284,2.5559,2.6541,2.7297,2.7881,2.8336,2.8694,2.898,2.4769,2.759,2.9384,3.0469,3.1077,3.1385,3.1521,3.1558,3.1554
2,"(2018, 1)",1.9119,2.1441,2.3127,2.4354,2.525,2.5909,2.6398,2.6766,2.705,2.7275,2.3763,2.6499,2.8035,2.8834,2.9204,2.9332,2.9342,2.9322,2.93
3,"(2017, 12)",1.7573,1.8852,2.0033,2.1056,2.1912,2.2613,2.318,2.3638,2.4012,2.432,2.0131,2.2395,2.4125,2.5336,2.6118,2.6582,2.6844,2.7004,2.7092
4,"(2017, 11)",1.6313,1.7902,1.9286,2.0468,2.1465,2.23,2.3,2.3589,2.4088,2.4518,1.9491,2.2054,2.4014,2.5453,2.6475,2.72,2.7712,2.808,2.8388
5,"(2017, 10)",1.4416,1.5965,1.7537,1.897,2.0211,2.1259,2.2133,2.2864,2.3481,2.4009,1.7514,2.0681,2.3269,2.5175,2.6499,2.7377,2.7981,2.8417,2.8761
6,"(2017, 9)",1.3341,1.481,1.6455,1.8015,1.939,2.0561,2.1541,2.236,2.305,2.3638,1.6279,1.9745,2.2695,2.489,2.6416,2.7421,2.8093,2.857,2.893


### (c)
$$rx_{t+1} = p_{t+1}(n-1) - p_t(n) - y_t(1)$$

In [5]:
pt = hcat([- n * data[:, 1+n] for n in 1:10]...)
rx_t1 = pt[1:end-1, 1:9] - pt[2:end, 2:10] - repmat(data[2:end, 2], 1, 9);

In [6]:
(rx_t1)

555×9 Array{Float64,2}:
 0.327    0.5      0.5445   0.4895  …   0.1885  -0.0158  -0.2357  -0.4615
 0.1012  -0.0356  -0.273   -0.5429     -1.0767  -1.3255  -1.5593  -1.7823
 0.1918   0.3841   0.546    0.6788      0.9009   1.0139   1.1375   1.2759
 0.1201   0.2391   0.3606   0.4767      0.6715   0.7496   0.8201   0.8882
 0.1863   0.4094   0.6108   0.7729      0.9892   1.0608   1.1197   1.171 
 0.0793   0.1753   0.2161   0.1913  …   0.0007  -0.1285  -0.2607  -0.3872
 0.2431   0.6935   1.1842   1.6474      2.4372   2.7738   3.0858   3.3792
 0.2901   0.7413   1.1478   1.4512      1.7646   1.8143   1.8189   1.7897
 0.1185   0.3892   0.6248   0.7708      0.8416   0.8265   0.8129   0.818 
 0.2488   0.7568   1.2218   1.5887      2.099    2.2888   2.4542   2.6051
 0.3844   1.0104   1.6034   2.0749  …   2.6684   2.8472   2.9825   3.0958
 0.5019   1.0853   1.4779   1.6867      1.8046   1.8057   1.7996   1.796 
 0.6074   1.1429   1.5681   1.9034      2.4159   2.631    2.8323   3.0224
 ⋮            

In [7]:
rxbar_t1 = 1/9 * sum(rx_t1, 2)
X = hcat(ones(length(rxbar_t1)), data[2:end, :SVENY01], [data[2:end, Symbol("fwd$(y)y")] for y in 2:5]...)

555×6 Array{Float64,2}:
 1.0  1.9119  2.3763  2.6499  2.8035  2.8834
 1.0  1.7573  2.0131  2.2395  2.4125  2.5336
 1.0  1.6313  1.9491  2.2054  2.4014  2.5453
 1.0  1.4416  1.7514  2.0681  2.3269  2.5175
 1.0  1.3341  1.6279  1.9745  2.2695  2.489 
 1.0  1.2322  1.4134  1.7239  2.0153  2.2447
 1.0  1.2383  1.4753  1.8638  2.2146  2.4785
 1.0  1.2608  1.5284  1.9265  2.2703  2.518 
 1.0  1.1887  1.3793  1.7991  2.1621  2.4163
 1.0  1.1191  1.4375  1.8873  2.2641  2.529 
 1.0  1.0551  1.5035  2.0635  2.4803  2.7356
 1.0  0.9164  1.557   2.0869  2.4561  2.6891
 1.0  0.8732  1.5238  2.0925  2.5121  2.7914
 ⋮                                    ⋮     
 1.0  5.4437  6.2401  6.3762  6.3992  6.4033
 1.0  5.4438  6.1952  6.4045  6.4633  6.4792
 1.0  5.3263  6.1211  6.396   6.491   6.5241
 1.0  4.7773  6.0205  6.2731  6.3243  6.3343
 1.0  5.0769  5.9409  6.1926  6.266   6.2876
 1.0  4.569   5.5524  6.0396  6.2814  6.4011
 1.0  4.5459  5.8073  6.2251  6.3637  6.4095
 1.0  5.0208  6.1222  6.3014  6

In [8]:
γ = (X' * X) \ (X' * rxbar_t1)

6×1 Array{Float64,2}:
  -0.370198
  -1.941   
   5.4919  
 -10.7566  
   7.98636 
  -0.739936

In [9]:
CP = (X * γ)[:, 1]

555-element Array{Float64,1}:
 0.721465
 0.577565
 0.740145
 0.925075
 1.02497 
 0.890991
 1.13293 
 1.12205 
 1.02468 
 1.26188 
 1.42704 
 1.5795  
 1.79233 
 ⋮       
 1.1155  
 1.02007 
 1.12062 
 1.76485 
 1.18103 
 1.71803 
 1.81846 
 1.5975  
 2.14449 
 2.12131 
 1.79862 
 1.45012 

$$CP_t = \hat{\overline{rx}}_{t+1}$$

### (e)

In [10]:
X = hcat(ones(length(CP)), CP)
f⟂ = hcat([data[2:end, Symbol("fwd$(n)y")] - X * ((X' * X) \ (X' * data[2:end, Symbol("fwd$(n)y")])) for n in 2:10]...);

### (f)

In [11]:
decomp = eig(cov(f⟂))
PC = f⟂ * decomp[2][:, end:-1:end-1] ./ sum(decomp[2][:, end:-1:end-1], 1)

555×2 Array{Float64,2}:
 -4.17463    -6.40323
 -4.56306    -5.39532
 -4.4493     -4.96229
 -4.3967     -4.49614
 -4.37884    -4.16745
 -4.66447    -3.6547 
 -4.33696    -3.43469
 -4.33279    -4.05077
 -4.47523    -3.38242
 -4.25409    -3.81175
 -4.03086    -4.49519
 -3.97295    -5.2773 
 -3.7669     -4.82464
  ⋮                  
 -0.402889   -5.55606
 -0.394712   -4.81071
 -0.317302   -4.61024
 -0.166146   -7.08932
 -0.51479    -5.27216
 -0.190327   -4.34138
 -0.109487   -6.15009
 -0.234283   -6.8893 
  0.0693883  -5.24443
  0.0740625  -5.75692
 -0.315897   -3.38269
 -0.390235   -3.63671

### (g)

In [12]:
CP = CP - mean(CP)
PC = PC .- mean(PC, 1)

555×2 Array{Float64,2}:
 -4.17463    -6.40323
 -4.56306    -5.39532
 -4.4493     -4.96229
 -4.3967     -4.49614
 -4.37884    -4.16745
 -4.66447    -3.6547 
 -4.33696    -3.43469
 -4.33279    -4.05077
 -4.47523    -3.38242
 -4.25409    -3.81175
 -4.03086    -4.49519
 -3.97295    -5.2773 
 -3.7669     -4.82464
  ⋮                  
 -0.402889   -5.55606
 -0.394712   -4.81071
 -0.317302   -4.61024
 -0.166146   -7.08932
 -0.51479    -5.27216
 -0.190327   -4.34138
 -0.109487   -6.15009
 -0.234283   -6.8893 
  0.0693883  -5.24443
  0.0740625  -5.75692
 -0.315897   -3.38269
 -0.390235   -3.63671

### (h)
#### (i)

In [13]:
X = hcat(ones(length(rxbar_t1)), data[2:end, :SVENY01], [data[2:end, Symbol("fwd$(y)y")] for y in 2:5]...)
γ = (X' * X) \ (X' * rxbar_t1)
1 - sum((rxbar_t1 - (X * γ)[:, 1]).^2)/sum((rxbar_t1 - mean(rxbar_t1)).^2)

0.43734213071923

#### (ii)
Until 1992.12 the $R^2$ is:

In [14]:
X = hcat(ones(length(rxbar_t1[218:end])), data[219:end, :SVENY01], [data[219:end, Symbol("fwd$(y)y")] for y in 2:5]...)
γ = (X' * X) \ (X' * rxbar_t1[218:end])
1 - sum((rxbar_t1[218:end] - (X * γ)[:, 1]).^2)/sum((rxbar_t1[218:end] - mean(rxbar_t1[218:end])).^2)

0.3581029321663768

From 1993 on the $R^2$ is:

In [15]:
X = hcat(ones(length(rxbar_t1[1:217])), data[2:218, :SVENY01], [data[2:218, Symbol("fwd$(y)y")] for y in 2:5]...)
γ = (X' * X) \ (X' * rxbar_t1[1:217])
1 - sum((rxbar_t1[1:217] - (X * γ)[:, 1]).^2)/sum((rxbar_t1[1:217] - mean(rxbar_t1[1:217])).^2)

0.5602113742763926

TODO: Interpret??

## Task 3

In [16]:
X = hcat(CP, PC)

555×3 Array{Float64,2}:
 -0.732341    -4.17463    -6.40323
 -0.876241    -4.56306    -5.39532
 -0.713662    -4.4493     -4.96229
 -0.528732    -4.3967     -4.49614
 -0.428833    -4.37884    -4.16745
 -0.562815    -4.66447    -3.6547 
 -0.320873    -4.33696    -3.43469
 -0.331753    -4.33279    -4.05077
 -0.429128    -4.47523    -3.38242
 -0.191922    -4.25409    -3.81175
 -0.0267684   -4.03086    -4.49519
  0.125697    -3.97295    -5.2773 
  0.338521    -3.7669     -4.82464
  ⋮                               
 -0.338307    -0.402889   -5.55606
 -0.433735    -0.394712   -4.81071
 -0.333187    -0.317302   -4.61024
  0.31104     -0.166146   -7.08932
 -0.272781    -0.51479    -5.27216
  0.264221    -0.190327   -4.34138
  0.364653    -0.109487   -6.15009
  0.143692    -0.234283   -6.8893 
  0.690681     0.0693883  -5.24443
  0.667508     0.0740625  -5.75692
  0.344815    -0.315897   -3.38269
 -0.00368774  -0.390235   -3.63671

In [17]:
y1 = data[2:end, :SVENY01]

555-element Array{Float64,1}:
 1.9119
 1.7573
 1.6313
 1.4416
 1.3341
 1.2322
 1.2383
 1.2608
 1.1887
 1.1191
 1.0551
 0.9164
 0.8732
 ⋮     
 5.4437
 5.4438
 5.3263
 4.7773
 5.0769
 4.569 
 4.5459
 5.0208
 4.2036
 4.243 
 4.3542
 4.6539

In [18]:
Z = hcat(ones(size(X, 1)), X)
δ = (Z' * Z) \ (Z' * y1)

4-element Array{Float64,1}:
  5.22048  
 -1.39617  
  1.04331  
 -0.0083798

In [19]:
δ1 = δ[2:4];

In [20]:
1 - sum((y1 - (Z * δ)[:, 1]).^2)/sum((y1 - mean(y1)).^2)

0.9939130606952067

I guess it's correct?!

## Task 4

In [21]:
Γ = ((X[13:end, :])' * X[13:end, :]) \ ((X[13:end, :])' * X[1:end-12, :])

3×3 Array{Float64,2}:
  0.540006   -0.492155    0.0659785 
 -0.0061813   0.900161   -0.00301876
  0.0888033   0.0906404   0.422987  

naive OLS estimate of $\Sigma$:

In [22]:
ϵ = ((X[1:end-12, :])' .- Γ * (X[13:end, :])')'
Σ = ϵ'ϵ/(size(ϵ, 1) - 9 - 1)

3×3 Array{Float64,2}:
  3.36613    0.640603  -0.810318
  0.640603   1.89841   -0.157742
 -0.810318  -0.157742   8.74069 

HAC estimate of the variance matrix of $vec(\Gamma)$:

In [23]:
function Shat_HAC(Y, X, β; Jn = 4)
    N = length(β)
    
    Γ = zeros(N, N)
    Γ!(Γ, Y, X, β, j = 0)
    
    #0th lag
    Shat = copy(Γ)
    
    #all other lags
    for j in 1:Jn
        Γ .= zeros(N, N)
        Γ!(Γ, Y, X, β, j = j)
        Shat .+= (1 - j / (Jn + 1)) .* (Γ .+ Γ')
    end
    
    return Shat
end

function Γ!(Γ, Y, X, β; j = 0)
    for t in j+1:size(X, 1)
        Γ .+= vec((Y[t, :] - β  * X[t, :]) * X[t, :]') * vec((Y[t-j, :] - β  * X[t-j, :]) * X[t-j, :]')'
    end
    
    Γ .= Γ ./ (size(X, 1) - j)
end

Γ! (generic function with 1 method)

In [24]:
round.(Shat_HAC(X[1:end-12, :], X[13:end, :], Γ, Jn = 12)/size(X, 1), 3)

9×9 Array{Float64,2}:
  0.097   0.034  -0.044  -0.054  -0.053   0.065  -0.046  -0.023   0.026
  0.034   0.106  -0.001  -0.084  -0.031  -0.018   0.003  -0.097  -0.037
 -0.044  -0.001   0.347   0.075  -0.014  -0.182   0.034  -0.052  -0.07 
 -0.054  -0.084   0.075   1.11   -0.037  -0.402   0.039   0.053  -0.096
 -0.053  -0.031  -0.014  -0.037   0.238   0.073   0.049   0.03    0.074
  0.065  -0.018  -0.182  -0.402   0.073   0.759   0.005   0.092   0.144
 -0.046   0.003   0.034   0.039   0.049   0.005   0.501   0.236  -0.23 
 -0.023  -0.097  -0.052   0.053   0.03    0.092   0.236   0.406  -0.058
  0.026  -0.037  -0.07   -0.096   0.074   0.144  -0.23   -0.058   0.81 

## Task 5

In [25]:
β = [begin
    X = hcat(ones(size(ϵ, 1)), ϵ[:, i])
    β = ((X' * X) \ (X' * rx_t1[13:end, :]))'
    β = β[:, 2]
end for i in 1:3]

result = [begin 
    λ = (β[i]' * β[i]) \ (β[i]' * (mean(rx_t1[13:end, :], 1) + 1/2 * var(rx_t1[13:end, :], 1))')
    (λ, (mean(rx_t1[13:end, :], 1) + 1/2 * var(rx_t1[13:end, :], 1))' - β[i] * λ)
end for i in 1:3];

In [26]:
λ = [result[i][1][1] for i in 1:3]
α = hcat([result[i][2] for i in 1:3]...)
table = DataFrame(hcat(string.(2:10), α), [:n, :CP, :level, :slope])
push!(table, vcat("MAE", mean(abs.(α), 1)[:]))
table

,n,CP,level,slope
1,2,-2.91725,-0.919833,-2.9727
2,3,-3.1289,-1.05425,-3.72931
3,4,-2.1432,-0.979099,-2.71145
4,5,-0.663953,-0.789444,-1.03596
5,6,0.9988,-0.515339,0.624124
6,7,2.68584,-0.172822,1.94819
7,8,4.31638,0.231316,2.83049
8,9,5.84925,0.698022,3.27982
9,10,7.27312,1.23323,3.36293
10,MAE,3.33074,0.732596,2.49944


The level factor really explains the best!

## Task 6
$$p_{t}(1)=-y_{t}(1)=\underset{A(1)}{\underbrace{-\delta_{0}}}\underset{B(1)}{\underbrace{-\delta_{1}}}X_{t}$$
The Euler equation for investing into a bond with maturity $2$ is:
$$\mathbb{E}_t\Big[ M_{t+1} \cdot P_{t+1}(1) \Big] = P_{t}(2)$$
Hence:
$$log\Big(\mathbb{E}_{t}\Big[M_{t+1}\cdot P_{t+1}(1)\Big]\Big)=log\Big(\mathbb{E}_{t}^{*}\Big[exp\big(p_{t+1}(1)\big)\Big]\Big)=log\Big(\mathbb{E}_{t}^{*}\Big[exp\big(-\delta_{0}-\delta_{1}X_{t+1}\big)\Big]\Big)=$$
$$-\delta_{0}+log\Big(\mathbb{E}_{t}^{*}\Big[exp\big(\underset{\sim\mathbb{\mathcal{N}\big(}-\delta_{1}\mu^{*}-\delta_{1}\Gamma^{*}X_{t},\delta_{1}^{'}\Sigma\delta_{1}\big)}{\underbrace{-\delta_{1}X_{t+1}}}\big)\Big]\Big)=\underset{=A(2)=A(1)-\delta_{1}\mu^{*}+\frac{1}{2}\delta_{1}^{'}\Sigma\delta_{1}}{\underbrace{-\delta_{0}-\delta_{1}\mu^{*}+\frac{1}{2}\delta_{1}^{'}\Sigma\delta_{1}}}\underset{B(2)=B(1)\Gamma^{*}}{\underbrace{-\delta_{1}\Gamma^{*}}}X_{t}=p_{t}(2)$$

Iterating one step further:
$$log\Big(\mathbb{E}_{t}\Big[M_{t+1}\cdot P_{t+1}(2)\Big]\Big)=log\Big(\mathbb{E}_{t}^{*}\Big[P_{t+1}(2)\Big]\Big)=-\delta_{0}-\delta_{1}\mu^{*}+\frac{1}{2}\delta_{1}^{'}\Sigma\delta_{1}-\delta_{1}\Gamma^{*}\mu^{*}+\frac{1}{2}\delta_{1}^{'}\Gamma^{*'}\Sigma\Gamma^{*}\delta_{1}-\delta_{1}\Gamma^{*}\Gamma^{*}X_{t}=p_{t}(3)$$

Hence:
$$B(n) = -\delta_1\Big(\Gamma^*\Big)^n$$
$$A(n) = -\delta_0 - \delta_1\Big(\sum_{i=1}^n \big(\Gamma^*\big)^{i-1}\Big)\mu^* + \frac{1}{2}\Big(\sum_{i=1}^n \delta_1^{'} \big(\Gamma^{*'}\big)^{i-1}\Sigma\big(\Gamma^*\big)^{i-1} \delta_1 \Big)$$

## Task 7

In [27]:
X = hcat(ones(size(CP[13:end, :], 1)), CP[13:end, :])
θ = ((X' * X) \ (X' * rx_t1[1:end-12, 9]))[2]

0.8460776092992276

In [28]:
Γstar(λ121) = Γ - Σ * reshape([0, λ121, 0, 0, 0, 0, 0, 0, 0], 3, 3)

Γstar (generic function with 1 method)

In [29]:
B(n, λ121) = -δ1' * Γstar(λ121)^n

B (generic function with 1 method)

In [75]:
δ1

3-element Array{Float64,1}:
 -1.39617  
  1.04331  
 -0.0083798

In [84]:
Γstar(λ121)

3×3 Array{Float64,2}:
  3.42509   -0.492155    0.0659785 
  8.54368    0.900161   -0.00301876
 -0.621619   0.0906404   0.422987  

In [86]:
residual(λ121) = (B(9, λ121) * Σ * reshape([0, λ121, 0, 0, 0, 0, 0, 0, 0], 3, 3))[1] - θ

residual (generic function with 1 method)

In [96]:
using Roots
range = collect(-9:0.1:5)
plot(range, residual.(range))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -8 
 
 
 -6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 -1500000 
 
 
 -1000000 
 
 
 -500000 
 
 
 0 
 
 
 500000 
 
 
 1000000 
 
 
 1500000 
 
<polyline clip-path="url(#clip3302)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 69.6096,11.811 73.2575,95.1362 76.9054,163.815 80.5533,219.723 84.2012,264.543 87.8491,299.782 91.497,326.788 95.1449,346.761 98.7928,360.766 102.441,369.747 
 106.089,374.537 109.736,375.869 113.384,374.383 117.032,370.639 120.68,365.121 124.328,358.249 127.976,350.38 131.624,341.821 135.272,332.831 138.92,323.624 
 142.568,314.38 146.215,305.245 149.863,296.336 153.511,287.747 157.159,279.547 160.807,271.787 164.455,264.505 168.103,257.722 171.751,251.449 175.399,245.687 
 179.046,240.428 182.694,235.661 186.342,231.366 189.99,227.522 193.638,224.104 197.286,221.086 200.934,218.438 204.582,216.133 208.23,214.142 211.878,212.437 
 215.525,210.989 219.173,209.773 222.821,208.763 226.469,207.936 230.117,207.268 233.765,206.739 237.413,206.33 241.061,206.024 244.709,205.804 248.357,205.656 
 252.004,205.568 255.652,205.528 259.3,205.527 262.948,205.556 266.596,205.607 270.244,205.674 273.892,205.752 277.54,205.836 281.188,205.923 284.835,206.011 
 288.483,206.096 292.131,206.177 295.779,206.254 299.427,206.324 303.075,206.388 306.723,206.446 310.371,206.498 314.019,206.543 317.667,206.582 321.314,206.615 
 324.962,206.644 328.61,206.668 332.258,206.687 335.906,206.704 339.554,206.717 343.202,206.727 346.85,206.735 350.498,206.741 354.145,206.746 357.793,206.75 
 361.441,206.752 365.089,206.754 368.737,206.755 372.385,206.756 376.033,206.756 379.681,206.756 383.329,206.757 386.977,206.757 390.624,206.756 394.272,206.756 
 397.92,206.756 401.568,206.756 405.216,206.757 408.864,206.757 412.512,206.757 416.16,206.757 419.808,206.758 423.455,206.759 427.103,206.759 430.751,206.76 
 434.399,206.761 438.047,206.762 441.695,206.763 445.343,206.764 448.991,206.766 452.639,206.767 456.287,206.768 459.934,206.77 463.582,206.771 467.23,206.773 
 470.878,206.774 474.526,206.774 478.174,206.774 481.822,206.772 485.47,206.769 489.118,206.762 492.765,206.751 496.413,206.734 500.061,206.708 503.709,206.671 
 507.357,206.619 511.005,206.546 514.653,206.446 518.301,206.311 521.949,206.131 525.597,205.894 529.244,205.584 532.892,205.183 536.54,204.667 540.188,204.01 
 543.836,203.178 547.484,202.133 551.132,200.827 554.78,199.206 558.428,197.204 562.075,194.745 565.723,191.74 569.371,188.085 573.019,183.66 576.667,178.326 
 580.315,171.92 
 "/>
 
 
 
 
 y1

In [32]:
fzero(residual, -10., -8)

-8.725173046730454

In [33]:
fzero(residual, -6., -4)

-4.503701780676638

In [97]:
λ121 = fzero(residual, -1., 1.)

-0.5497627981006247

There are three $\lambda_{1(2,1)}$ that equate $\theta = B(n-1)^{'} \Sigma \lambda_1$.

## Task 8

In [106]:
residual(λ02) = (mean(rx_t1) + 1/2*B(9, λ121) * Σ * B(9, λ121)' - B(9, λ121) * Σ * reshape([0, λ02, 0], 3, 1))[1]

residual (generic function with 1 method)

In [109]:
range = collect(-50:0.1:0)
plot(range, residual.(range))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -50 
 
 
 -40 
 
 
 -30 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
 
 -20 
 
 
 0 
 
 
 20 
 
 
 40 
 
<polyline clip-path="url(#clip4702)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 36.1567,375.869 37.245,375.141 38.3334,374.412 39.4217,373.684 40.51,372.956 41.5983,372.228 42.6866,371.5 43.7749,370.772 44.8633,370.044 45.9516,369.316 
 47.0399,368.588 48.1282,367.859 49.2165,367.131 50.3048,366.403 51.3932,365.675 52.4815,364.947 53.5698,364.219 54.6581,363.491 55.7464,362.763 56.8347,362.034 
 57.9231,361.306 59.0114,360.578 60.0997,359.85 61.188,359.122 62.2763,358.394 63.3646,357.666 64.453,356.938 65.5413,356.21 66.6296,355.481 67.7179,354.753 
 68.8062,354.025 69.8945,353.297 70.9828,352.569 72.0712,351.841 73.1595,351.113 74.2478,350.385 75.3361,349.657 76.4244,348.928 77.5127,348.2 78.6011,347.472 
 79.6894,346.744 80.7777,346.016 81.866,345.288 82.9543,344.56 84.0426,343.832 85.131,343.103 86.2193,342.375 87.3076,341.647 88.3959,340.919 89.4842,340.191 
 90.5725,339.463 91.6609,338.735 92.7492,338.007 93.8375,337.279 94.9258,336.55 96.0141,335.822 97.1024,335.094 98.1908,334.366 99.2791,333.638 100.367,332.91 
 101.456,332.182 102.544,331.454 103.632,330.726 104.721,329.997 105.809,329.269 106.897,328.541 107.986,327.813 109.074,327.085 110.162,326.357 111.251,325.629 
 112.339,324.901 113.427,324.172 114.516,323.444 115.604,322.716 116.692,321.988 117.78,321.26 118.869,320.532 119.957,319.804 121.045,319.076 122.134,318.348 
 123.222,317.619 124.31,316.891 125.399,316.163 126.487,315.435 127.575,314.707 128.664,313.979 129.752,313.251 130.84,312.523 131.929,311.795 133.017,311.066 
 134.105,310.338 135.194,309.61 136.282,308.882 137.37,308.154 138.458,307.426 139.547,306.698 140.635,305.97 141.723,305.241 142.812,304.513 143.9,303.785 
 144.988,303.057 146.077,302.329 147.165,301.601 148.253,300.873 149.342,300.145 150.43,299.417 151.518,298.688 152.607,297.96 153.695,297.232 154.783,296.504 
 155.872,295.776 156.96,295.048 158.048,294.32 159.136,293.592 160.225,292.864 161.313,292.135 162.401,291.407 163.49,290.679 164.578,289.951 165.666,289.223 
 166.755,288.495 167.843,287.767 168.931,287.039 170.02,286.31 171.108,285.582 172.196,284.854 173.285,284.126 174.373,283.398 175.461,282.67 176.55,281.942 
 177.638,281.214 178.726,280.486 179.814,279.757 180.903,279.029 181.991,278.301 183.079,277.573 184.168,276.845 185.256,276.117 186.344,275.389 187.433,274.661 
 188.521,273.933 189.609,273.204 190.698,272.476 191.786,271.748 192.874,271.02 193.963,270.292 195.051,269.564 196.139,268.836 197.228,268.108 198.316,267.379 
 199.404,266.651 200.493,265.923 201.581,265.195 202.669,264.467 203.757,263.739 204.846,263.011 205.934,262.283 207.022,261.555 208.111,260.826 209.199,260.098 
 210.287,259.37 211.376,258.642 212.464,257.914 213.552,257.186 214.641,256.458 215.729,255.73 216.817,255.002 217.906,254.273 218.994,253.545 220.082,252.817 
 221.171,252.089 222.259,251.361 223.347,250.633 224.435,249.905 225.524,249.177 226.612,248.449 227.7,247.72 228.789,246.992 229.877,246.264 230.965,245.536 
 232.054,244.808 233.142,244.08 234.23,243.352 235.319,242.624 236.407,241.895 237.495,241.167 238.584,240.439 239.672,239.711 240.76,238.983 241.849,238.255 
 242.937,237.527 244.025,236.799 245.113,236.071 246.202,235.342 247.29,234.614 248.378,233.886 249.467,233.158 250.555,232.43 251.643,231.702 252.732,230.974 
 253.82,230.246 254.908,229.518 255.997,228.789 257.085,228.061 258.173,227.333 259.262,226.605 260.35,225.877 261.438,225.149 262.527,224.421 263.615,223.693 
 264.703,222.964 265.791,222.236 266.88,221.508 267.968,220.78 269.056,220.052 270.145,219.324 271.233,218.596 272.321,217.868 273.41,217.14 274.498,216.411 
 275.586,215.683 276.675,214.955 277.763,214.227 278.851,213.499 279.94,212.771 281.028,212.043 282.116,211.315 283.205,210.587 284.293,209.858 285.381,209.13 


In [111]:
λ02 = fzero(residual, -50, 0)

-33.12155432842244